# Backtest

In [1]:
from system.agent import Agent
from system.env import Env

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2023", "end" : "2023"
          }


env = Env(**config)

params_1 = {"symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"MOMENTUM"}
params_2 = {"symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA"}

agent_1 = Agent(**params_1)
agent_1.update_policy_params(params = 3)

agent_2 = Agent(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))

In [2]:
agent_1.run_episode()
#agent_2.run_episode()

In [3]:
agent_1.get_report()

In [4]:
agent_2.run_episode()

In [5]:
agent_2.get_report()

In [6]:
env.journal.portfolio_data

,date,risk_value,available_value,capital,symbol
0,2023-02-14,100.000000,0.00000,100.000000,BTC
1,2023-02-15,109.568582,0.00000,109.568582,BTC
2,2023-02-16,105.936439,0.00000,105.936439,BTC
3,2023-02-17,110.676338,0.00000,110.676338,BTC
4,2023-02-18,110.718906,0.00000,110.718906,BTC
...,...,...,...,...,...
487,2023-10-13,0.000000,160.95507,160.955070,BTC
488,2023-10-14,0.000000,160.95507,160.955070,BTC
489,2023-10-15,0.000000,160.95507,160.955070,BTC
490,2023-10-16,0.000000,160.95507,160.955070,BTC
